PROJECT

Deliverables:
    Notebook
    Documentation of the model
    Test that everything works well
Goals:
    Compare the environmental performance of 2 biobased insulation materials contrasted to a reference fossil insulation      material, applied     to buildings throught the prodution and end-of-life phases of the product.
Scope:
    The environmental performance is assessed through the climate change (CO2, Ch4, N2O), water use, and land use (area occupied) impacts
    LCIA method: 
    The assessment is performed for 50 years of lifetime of the buildings
    The EoL choices involve incineration, landfill, composting, and mulching (we are going to consider 2)
    Temporal scope:
    
Tools
    Github
    Temporalis
    Excel

  *How to consider the soil carbon sequestration/release/lost effect
  For croplands is shor, for forests is large


Prerequisites:
    Python: nbdev and ipytest
Your environment must contain nbdev and nbdev_install_hooks
For this project:
Open conda terminal 
conda activate bw25
    This env already contains nbdev 

In [30]:
from bw_temporalis import easy_timedelta_distribution, easy_datetime_distribution, TemporalisLCA, Timeline, TemporalDistribution
#pip install dynamic_characterization
# from dynamic_characterization.ipcc_ar6.radiative_forcing import characterize_co2, characterize_ch4, characterize_n2o, characterize_co2_uptake
from bw_temporalis.lcia import characterize_methane, characterize_co2
import bw2io as bi 
import bw2data as bd
import bw2calc as bc
import os               # to use "operating system dependent functionality"
import numpy as np      # "the fundamental package for scientific computing with Python"
import pandas as pd     # "high-performance, easy-to-use data structures and data analysis tools" for Python

import bw_temporalis as bt

In [2]:
list(bd.projects)

[Project: default,
 Project: bw25_intro,
 Project: bw25,
 Project: bw25_intro2,
 Project: parameterLCA,
 Project: premise,
 Project: parametric_LCA,
 Project: Temporalis example project,
 Project: bw_temporalis example,
 Project: dds-MFA,
 Project: Buildings Insulation,
 Project: building,
 Project: buildings]

In [5]:
#Load ecoinvent and biosphere databases
#Either import from ecoinvent or load it from a file saved in your computer - Here we load from a filed saved from a previous project
# bi.backup.restore_project_directory(
#     fp=r"C:\Users\zieger\Github\brightway2-project-bw25_workgroup-backup14-November-2024-03-02PM.tar.gz",
#     project_name="Buildings Insulation",
#     overwrite_existing=True
# )

Restoring project backup archive - this could take a few minutes...
Restored project: Buildings Insulation


'Buildings Insulation'

In [3]:
bd.projects.set_current('building')

In [4]:
bd.databases

Databases dictionary with 9 object(s):
	PV 2035
	PV 2050
	PV 2050 - eff
	PV 2050 - eff - lifetime
	biosphere
	carbon fiber
	ecoinvent-3.10-cutoff
	example
	temporalis-example

In [6]:
imp = bi.ExcelImporter("LCI_Bw_form.xlsx")
imp.apply_strategies()

Extracted 1 worksheets in 0.02 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 4.99 seconds


In [7]:
imp.match_database('ecoinvent-3.10-cutoff', fields=('name', 'unit', 'location', 'reference product'))
imp.statistics()

Applying strategy: link_iterable_by_fields
4 datasets
	15 exchanges
	Links to the following databases:
		ecoinvent-3.10-cutoff (9 exchanges)
		Insulators (4 exchanges)
		biosphere (2 exchanges)
	0 unlinked exchanges (0 types)
		


(4, 15, 0, 0)

In [8]:
imp.write_database()

11:26:04 [warning  ] Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|███████████████████████████████████████████| 4/4 [00:00<00:00, 2412.95it/s]

11:26:05 [info     ] Vacuuming database            


Created database: Insulators


In [41]:
#imp.write_excel()

Wrote matching file to:
C:\Users\andraded\AppData\Local\pylca\Brightway3\Buildings-Insulation.2ffa2648\output\db-matching-Insulators.xlsx


WindowsPath('C:/Users/andraded/AppData/Local/pylca/Brightway3/Buildings-Insulation.2ffa2648/output/db-matching-Insulators.xlsx')

In [126]:
bd.databases

Databases dictionary with 10 object(s):
	Insulators
	PV 2035
	PV 2050
	PV 2050 - eff
	PV 2050 - eff - lifetime
	biosphere
	carbon fiber
	ecoinvent-3.10-cutoff
	example
	temporalis-example

In [9]:
Insulator_db = bd.Database('Insulators')

In [10]:
Insulator_db.random()

'Straw_bale_EoL' (unit, RER, None)

In [11]:
eidb=bd.Database('ecoinvent-3.10-cutoff')

In [12]:
bd.databases

Databases dictionary with 10 object(s):
	Insulators
	PV 2035
	PV 2050
	PV 2050 - eff
	PV 2050 - eff - lifetime
	biosphere
	carbon fiber
	ecoinvent-3.10-cutoff
	example
	temporalis-example

In [15]:
straw_prod = [
    act for act in Insulator_db if 'Straw_bale_product' in act['name']
][0]

In [29]:
for exc in straw_prod.exchanges():
    # print(exc)
    print(exc.input)
    

'Straw_bale_product' (unit, RER, None)
'wheat grain production, Swiss integrated production, intensive' (kilogram, CH, None)
'transport, freight, lorry 16-32 metric ton, EURO6' (ton kilometer, RER, None)
'market for electricity, low voltage' (kilowatt hour, FR, None)
'market for diesel' (kilogram, CH, None)
'Carbon dioxide, from soil or biomass stock' (kilogram, None, ('air',))


In [71]:
process_carbon = [exc for exc in straw_prod.exchanges()][-1]

In [72]:
process_carbon.as_dict()

{'name': 'Carbon dioxide, from soil or biomass stock',
 'amount': -52.8,
 'unit': 'kilogram',
 'categories': ('air',),
 'type': 'biosphere',
 'database': 'biosphere',
 'input': ('biosphere', 'e4e9febc-07c1-403d-8d3a-6707bb4d96e6'),
 'output': ('Insulators', '4d74c32f4972cb83fdafd0560027ceff'),
 'temporal_distribution': TemporalDistribution instance with 1 values and total: 1}

In [102]:
temporal_distribution_0 = TemporalDistribution(
                    np.array([0], dtype='timedelta64[Y]'),  # Yearly time steps
                    np.array([1])
                )

In [111]:
temporal_distribution_1 = TemporalDistribution(
                    np.array([1], dtype='timedelta64[Y]'),  # Yearly time steps
                    np.array([1])
                )

In [112]:
process_carbon['temporal_distribution'] = temporal_distribution_0

In [113]:
process_carbon.as_dict()

{'name': 'Carbon dioxide, from soil or biomass stock',
 'amount': -52.8,
 'unit': 'kilogram',
 'categories': ('air',),
 'type': 'biosphere',
 'database': 'biosphere',
 'input': ('biosphere', 'e4e9febc-07c1-403d-8d3a-6707bb4d96e6'),
 'output': ('Insulators', '4d74c32f4972cb83fdafd0560027ceff'),
 'temporal_distribution': TemporalDistribution instance with 1 values and total: 1}

In [114]:
[exc for exc in straw_prod.exchanges()][-1].as_dict()

{'name': 'Carbon dioxide, from soil or biomass stock',
 'amount': -52.8,
 'unit': 'kilogram',
 'categories': ('air',),
 'type': 'biosphere',
 'database': 'biosphere',
 'input': ('biosphere', 'e4e9febc-07c1-403d-8d3a-6707bb4d96e6'),
 'output': ('Insulators', '4d74c32f4972cb83fdafd0560027ceff'),
 'temporal_distribution': TemporalDistribution instance with 1 values and total: 1}

In [115]:
process_carbon.save()

In [116]:
process_wheat = [exc for exc in straw_prod.exchanges()][1]
process_wheat['temporal_distribution'] = temporal_distribution_0
process_wheat.save()
process_wheat.as_dict()

In [117]:
process_wheat.save()
process_wheat.as_dict()

{'name': 'wheat grain production, Swiss integrated production, intensive',
 'amount': 35,
 'reference product': 'straw',
 'location': 'CH',
 'unit': 'kilogram',
 'type': 'technosphere',
 'database': 'ecoinvent-3.10-cutoff',
 'input': ('ecoinvent-3.10-cutoff', 'ec93e72d405e4697085bad3af1c26c20'),
 'output': ('Insulators', '4d74c32f4972cb83fdafd0560027ceff'),
 'temporal_distribution': TemporalDistribution instance with 1 values and total: 1}

In [118]:
process_transport = [exc for exc in straw_prod.exchanges()][2]
process_transport['temporal_distribution'] = temporal_distribution_1
process_transport.save()
process_transport.as_dict()

{'name': 'transport, freight, lorry 16-32 metric ton, EURO6',
 'amount': 0.525,
 'reference product': 'transport, freight, lorry 16-32 metric ton, EURO6',
 'location': 'RER',
 'unit': 'ton kilometer',
 'type': 'technosphere',
 'database': 'ecoinvent-3.10-cutoff',
 'input': ('ecoinvent-3.10-cutoff', 'bc403494d398b1abe796db9820541f85'),
 'output': ('Insulators', '4d74c32f4972cb83fdafd0560027ceff'),
 'temporal_distribution': TemporalDistribution instance with 1 values and total: 1}

In [93]:
ipcc2021 = [m for m in bd.methods if 'IPCC' in str(m)
                    and ('2021') in str(m)
                    and 'GWP100' in str(m)
                    and 'no LT' not in str(m)
                    and 'SLCF' not in str(m)
           ][0]

In [94]:
ipcc_2021_method = bd.Method(ipcc2021)

In [119]:
lca = bc.LCA({straw_prod: 1}, ipcc_2021_method.name)
lca.lci()
lca.lcia()

tlca = TemporalisLCA(lca)
tl = tlca.build_timeline()
df = tl.build_dataframe()



/opt/anaconda3/envs/bw25/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 1.21e+13)
  warnings.warn(msg, UmfpackWarning)


Starting graph traversal
Calculation count: 31


In [120]:
df

,date,amount,flow,activity
0,2024-11-15 11:38:37,-5.280000e+01,3208,121996
1,2024-11-15 11:38:37,1.392793e-02,107,8505
2,2024-11-15 11:38:37,4.517530e-05,4192,23102
3,2024-11-15 11:38:37,1.200443e-06,4193,5497
4,2024-11-15 11:38:37,3.119262e-02,1031,5497
5,2024-11-15 11:38:37,5.028629e-07,4193,20327
6,2024-11-15 11:38:37,1.304419e-02,1031,20327
7,2024-11-15 11:38:37,2.032247e-06,1390,26435
8,2024-11-15 11:38:37,1.890462e-06,4193,26435
9,2024-11-15 11:38:37,4.867940e-02,1031,26435


In [128]:
df['cum_amount'] = df['amount'].cumsum()

df_modi = df.merge(
    bd.Database('Insulators').nodes_to_dataframe().rename(columns={'id': 'activity'}), 
    on='activity'
)

ValueError: The column label 'activity' is not unique.

In [123]:
bd.Database('Insulators').nodes_to_dataframe().rename(columns={'id': 'activity'})

,code,comment,database,activity,location,name,production amount,reference product,type,unit,worksheet name
0,9503250025219c27273ebe1b28e0ac7a,None,Insulators,121995,RER,EPS_EoL,1,"EPS, insulation",processwithreferenceproduct,unit,lci
3,10c1060a3a5c2d5bb0ea1149d9b8bf62,None,Insulators,121994,RER,EPS_product,1,"EPS, insulation",processwithreferenceproduct,unit,lci
1,9e8f6c61071e39dbf065233a99e0f8a4,None,Insulators,121997,RER,Straw_bale_EoL,1,"Straw_bale, insulation",processwithreferenceproduct,unit,lci
2,4d74c32f4972cb83fdafd0560027ceff,None,Insulators,121996,RER,Straw_bale_product,1,"Straw_bale, insulation",processwithreferenceproduct,unit,lci


In [122]:
df_modi

,date,amount,flow,activity,cum_amount,code,comment,database,location,name,production amount,reference product,type,unit,worksheet name
0,2024-11-15 11:38:37,-52.799999,3208,121996,-52.799999,4d74c32f4972cb83fdafd0560027ceff,None,Insulators,RER,Straw_bale_product,1,"Straw_bale, insulation",processwithreferenceproduct,unit,lci


In [97]:
characterized_df = tl.characterize_dataframe(
        characterization_function=characterize_co2, 
        flow={bd.get_node(name="carbon dioxide").id},  # Flow ID for CO2
        cumsum=True  # Cumulative sum of amounts
    )

# Filter out rows where the characterized amount is less than or equal to 0
characterized_df = characterized_df[characterized_df['amount'] > 0]

MultipleResults: Found 2 results for the given search

In [ ]:
df['c'] = df['amount'].cumsum()

df = df.merge(
    bd.Database('temporalis-example').nodes_to_dataframe().rename(columns={'id': 'activity'}), 
    on='activity'
)

In [44]:
# temporal_distribution = temporal_distribution': TemporalDistribution(
#                     np.array([0], dtype='timedelta64[Y]'),  # Yearly time steps
#                     np.array([1])
#                 )

# Add a new exchange with temporal distribution
exchange = {
    'amount': 20,  # Example emission amount
    'input': ('biosphere', 'carbon dioxide'),  # Reference to the CO2 emission flow
    'type': 'biosphere',  # Indicates it's a biosphere exchange
    'temporal_distribution': temporal_distribution
}

# Append the exchange to the process's exchanges
if 'exchanges' not in process:
    process['exchanges'] = []

process['exchanges'].append(exchange)

# Write the updated process back to the database
db.write()


NameError: name 'db' is not defined